<a href="https://colab.research.google.com/github/jwywoo/colab-practice/blob/feat%5D-%231-dataAcqusition/Day1_Data_acquisition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Web Crawler Import Setting

In [41]:
# requests
import requests
from bs4 import BeautifulSoup

## Requesting by request

In [42]:
# URL and response
url = 'https://www.aihub.or.kr'
response = requests.get(url)
response.raise_for_status()

## HTML Parsing By BeautifulSoup

In [43]:
# BeautifulSoup Object & HTML Parsing
soup = BeautifulSoup(response.content, 'html.parser')
top3_section = soup.find('div', class_='secR')
data_list = top3_section.find_all('div', class_='list')

titles = []
for data in data_list:
  title = data.find('h3').get_text(strip=True)
  titles.append(title.split(']')[-1].strip())

for index, title in enumerate(titles, start=1):
  print(f"TOP {index}: {title}")

TOP 1: 객체 간 관계성 인지용 한국형 비전 데이터
TOP 2: 손∙팔 협조에 의한 파지-조작 동작 데이터
TOP 3: 상용 자율주행차 야간 자동차 전용도로 데이터


In [44]:
url = 'https://ko.wikipedia.org/wiki/위키백과:대문'
response = requests.get(url)

if response.status_code == 200:
  soup = BeautifulSoup(response.content,'html.parser')
  title = soup.find('h1', id='firstHeading').text
  print(f"Title: {title}")
  first_paragraph = soup.find('p').text
  print(f"First paragraph: {first_paragraph}")
else:
  print(f"Filed to retrieve the web page. Status code: {response.status_code}")

Title: 위키백과:대문
First paragraph:  위키백과



In [45]:
!pip install selenium

## Selenium: For web-scraping but time consuming and not a good idea

### import to use selenium

In [46]:
from selenium import webdriver
from selenium.webdriver.common.by import By

In [47]:
URL = "https://ko.wikipedia.org/wiki/위키백과:대문"

# Chrome Options
options = webdriver.ChromeOptions()
options.add_argument("--headless") # 브라우저 창을 띄우지 않음
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--no-sandbox")

# webdriver setting
driver = webdriver.Chrome(options=options)

try:
  # opening wikipedia
  driver.get(URL)
  main_content = driver.find_element(By.CSS_SELECTOR, "#mw-content-text > div.mw-content-ltr.mw-parser-output > div.main-box.main-top > div > p:nth-child(2)").text
  print("Main Content", main_content)
finally:
  driver.quit()



Main Content 우리 모두가 만들어가는 자유 백과사전
문서 674,694개와 최근 기여자 1,814명


## Scrapy: Webscraping


In [48]:
!pip install scrapy

### Scrapy START


In [49]:
!scrapy startproject wikipedia_scraper

Error: scrapy.cfg already exists in /content/wikipedia_scraper


### Scrapy import and Class

In [50]:
import scrapy

class WikipediaSpider(scrapy.Spider):
  name = "wikipedia"
  start_urls = [
        'https://ko.wikipedia.org/wiki/위키백과:대문',
    ]
  def parse(self, response):
        main_content = response.css('#mw-content-text > div.mw-content-ltr.mw-parser-output > div.main-pane > div.main-pane-right > div.wikipedia-ko.main-recommended.main-box').get()
        yield {
            'main_content': main_content,
        }

In [51]:
%%writefile -a wikipedia_scraper/wikipedia_scraper/settings.py
ROBOTSTXT_OBEY = False

Appending to wikipedia_scraper/wikipedia_scraper/settings.py


In [62]:
from scrapy.crawler import CrawlerProcess
# not working
# process = CrawlerProcess({
#     'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
#     'FEEDS': {
#         'output.json': {
#             'format': 'json',
#             'encoding': 'utf8',
#             'store_empty': False,
#             'fields': None,
#             'indent': 4,
#         },
#     },
# })

# process.crawl(WikipediaSpider)
# process.start()